## Importing Libraries

In [22]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

#import library to parse the html table
!pip install Beautifulsoup4
!pip install lxml
!pip install requests

from bs4 import BeautifulSoup
import urllib.request

print('Libraries imported.')

Libraries imported.


## Parse the web content

In [78]:
datasetSource = urllib.request.urlopen('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').read()
soup = BeautifulSoup(datasetSource, 'lxml')
#Get the first table in the wiki page, that contains the desired dataset
table = soup.find_all('table')[0]
dfs = pd.read_html(str(table))
df = dfs[0]

#Clean the dataset
df_dataset = df[df.Borough != 'Not assigned']
df_dataset.loc[df.Neighbourhood=='Not assigned', 'Neighbourhood'] = df_dataset.Borough

#print a sample of the dataframe
df_dataset.head(15)

/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.


,Postcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M6A,North York,Lawrence Heights
6,M6A,North York,Lawrence Manor
7,M7A,Downtown Toronto,Queen's Park
9,M9A,Queen's Park,Queen's Park
10,M1B,Scarborough,Rouge
11,M1B,Scarborough,Malvern
13,M3B,North York,Don Mills North


## Get the Latitude & Longitude data



In [84]:
#get the spatial dataset and then merge with the previous dataset to get the full dataset
lon_lat_data = pd.read_csv("http://cocl.us/Geospatial_data")
lon_lat_data.columns = ['Postcode','Latitude','Longitude']
lon_lat_data.head(5)

,Postcode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [99]:
#identify indices
lon_lat_data.index = lon_lat_data['Postcode']
df_dataset.index = df_dataset['Postcode']
#join the two datasets on the Postcode column
df_dataset_full = pd.concat([df_dataset, lon_lat_data], axis=1, join='inner')
#reset index and remove the duplicate column after the merge
df_dataset_full.reset_index(drop=True, inplace=True)
df_dataset_full = df_dataset_full.loc[:,~df_dataset_full.columns.duplicated()]
df_dataset_full.head(5)

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,Harbourfront,43.654260,-79.360636
3,M6A,North York,Lawrence Heights,43.718518,-79.464763
4,M6A,North York,Lawrence Manor,43.718518,-79.464763
